<a href="https://colab.research.google.com/github/pratikunterwegs/duke-squirrel/blob/master/04_landcover_cortisol_duke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installs geemap package
import subprocess
import os

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [13]:
# prelim code for earth engine landcover around duke

# get transects from user assets
transects = ee.FeatureCollection("users/pratik_unterwegs/data_cortisol_buffer")
extent = ee.FeatureCollection("users/pratik_unterwegs/data_bboxes_cortisol")

# get buffer
extent_geom = extent.geometry()
extent_geom = extent_geom.buffer(5000)

# define start and end date
start_date = '2020-04-01'
end_date = '2020-10-01'

# filter sentinel by bounding box
# get sentinel data
sentinel = ee.ImageCollection("COPERNICUS/S2")
sentinel_bounds = sentinel.filterBounds(extent_geom)
sentinel_2020 = sentinel_bounds.filterDate(start_date, end_date)

# adding a NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands([ndvi])


# add an NDBI band
def add_ndbi(image):
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('ndbi')
    return image.addBands([ndbi])


# map metrics over sentinel
sentinel_ndvi = sentinel_2020.map(add_ndvi)
sentinel_ndbi = sentinel_2020.map(add_ndbi)

# get median values
ndvi_median = sentinel_ndvi.select('ndvi').median()
ndbi_median = sentinel_ndbi.select('ndbi').median()

# clip to geometry
ndvi_clip = ndvi_median.clip(extent_geom)
ndbi_clip = ndbi_median.clip(extent_geom)

In [ ]:
# Define an sld style color ramp to apply to the image.
sld_ramp = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="ramp" extended="false" >' + \
      '<ColorMapEntry color="#0D0887" quantity="-1" label="0"/>' + \
      '<ColorMapEntry color="#CC4678" quantity="0" label="300" />' + \
      '<ColorMapEntry color="#F0F921" quantity="0.5" label="500" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>';

# print layers
vis = {'bands': ['ndvi']}
Map = geemap.Map(center=[36.0005,-78.9], zoom=12)
Map.addLayer(ndbi_clip.sldStyle(sld_ramp), {}, 'builtup')
Map.addLayer(transects)
Map.addLayerControl()

Map

In [9]:
# check working directory
os.getcwd()

'/home/pratik/projects/stress-squirrel'

In [ ]:
out_dir = os.path.join(os.getcwd(), 'data')
# for ndvi
out_ndvi_mean = os.path.join(out_dir, 'data_zonalstats_ndvi_mean.csv')
out_ndvi_std = os.path.join(out_dir, 'data_zonalstats_ndvi_std.csv')

# for ndbi
out_ndbi_mean = os.path.join(out_dir, 'data_zonalstats_ndbi_mean.csv')
out_ndbi_std = os.path.join(out_dir, 'data_zonalstats_ndbi_std.csv') 

# check for directory and make
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
# for ndvi
geemap.zonal_statistics(ndvi_clip, transects, out_ndvi_mean, statistics_type='MEAN', scale=10)
geemap.zonal_statistics(ndvi_clip, transects, out_ndvi_std, statistics_type='STD', scale=10)

# for ndbi
geemap.zonal_statistics(ndbi_clip, transects, out_ndbi_mean, statistics_type='MEAN', scale=10)
geemap.zonal_statistics(ndbi_clip, transects, out_ndbi_std, statistics_type='STD', scale=10)

In [14]:
# to export image by area
featlist = extent.getInfo()["features"]

featlist

[{'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-79.06210018487616, 35.89912116447075],
     [-79.00099043638997, 35.89912115730096],
     [-78.93988066496506, 35.89912119024453],
     [-78.93988066496506, 36.05236713526263],
     [-79.00099043638997, 36.05236713957992],
     [-79.06210018487616, 36.05236714533879],
     [-79.06210018487616, 35.89912116447075]]]},
  'id': '00000000000000000000',
  'properties': {'level_0': 0, 'level_1': 0}},
 {'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-80.34929994709324, 43.38014167658449],
     [-80.29955421254233, 43.38014162011698],
     [-80.24980850186112, 43.3801416621483],
     [-80.20006271912598, 43.38014163570123],
     [-80.15031702319722, 43.38014161444812],
     [-80.15031702319722, 43.54981487095105],
     [-80.20006271912598, 43.549814915318414],
     [-80.24980850186112, 43.54981488107669],
     [-80.29955421254233, 43.549814912839565],
     [-80.34929994709324, 43.549814886342

In [12]:
# to export image by area
featlist = extent.getInfo()["features"]

#print featlist

def unpack(thelist):
    unpacked = []
    for i in thelist:
        unpacked.append(i[0])
        unpacked.append(i[1])
        # unpacked.append(i[2])
    return unpacked

for f, x in zip(featlist, [0, 1, 2]):
    print("feature = " + str(x))
    geomlist = unpack(f["geometry"]["coordinates"][0])
    geom = ee.Geometry.Polygon(geomlist)

    feat = ee.Feature(geom)
    feat = feat.buffer(3000)
    # disS = f["properties"]["district"]
    name_ndbi = "raster_ndbi_clip" + "_area_" + str(x)
    name_ndvi = "raster_ndvi_clip" + "_area_" + str(x)

    task_ndbi = ee.batch.Export.image.toDrive(
        image=ndbi_clip,
        # description= name+str(disS),
        folder= "stress-squirrel",
        fileNamePrefix= name_ndbi,
        region= feat.geometry().bounds().getInfo()["coordinates"],
        scale= 10
    )
    task_ndvi = ee.batch.Export.image.toDrive(
        image=ndvi_clip,
        folder= "stress-squirrel",
        fileNamePrefix= name_ndvi,
        region= feat.geometry().bounds().getInfo()["coordinates"],
        scale= 10
    )

    task_ndbi.start()
    task_ndvi.start()
    # print("exporting {0} {1}..".format(name))
